In [1]:
import tkinter as tk
from tkinter import messagebox
import os
import time
import glob
import shutil
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

# --- Windows 환경에서만 pywin32 import 및 사용 ---
IS_WINDOWS = os.name == 'nt'
if IS_WINDOWS:
    try:
        import win32gui
        import win32con
        print("pywin32 라이브러리를 성공적으로 로드했습니다.")
    except ImportError:
        print("경고: pywin32 라이브러리가 설치되지 않았습니다. 'pip install pywin32'를 실행하여 설치하세요.")
        IS_WINDOWS = False # pywin32 로드 실패 시 기능 비활성화

# --- Tkinter 고해상도(High DPI) 지원 시도 ---
if IS_WINDOWS:
    try:
        from ctypes import windll
        windll.shcore.SetProcessDpiAwareness(1)
        print("DPI 인식 설정을 시도했습니다.")
    except:
        print("DPI 인식 설정 중 오류가 발생했거나 지원되지 않는 환경입니다.")
        pass

# --- 설정 변수 ---
WINDOW_WIDTH = 500   # 창 너비
WINDOW_HEIGHT = 300  # 창 높이
BG_COLOR = "white"
DOWNLOAD_DIR = r"C:\Bi시각화\시계열" # 다운로드 폴더 경로

KB_BASE_URL = "https://kbland.kr/webview.html#/main/statistics?channel=kbland"
# 최신 확인된 주간시계열 탭 XPATH 사용 (혹시 변경되었는지 다시 한번 확인 필요)
KB_WEEKLY_TAB_XPATH = '//*[@id="__BVID__30___BV_tab_button__"]'
# 주간, 월간 다운로드 버튼 XPATH가 동일하다고 가정 (혹시 변경되었는지 다시 한번 확인 필요)
DOWNLOAD_BUTTON_XPATH = '//*[@id="reference2"]/div[1]/button'

# 웹 페이지 초기 로딩 및 탭 활성화 대기 시간 (3초 유지)
INITIAL_LOAD_WAIT = 3
TAB_LOAD_WAIT = 3


# --- 필수 라이브러리 확인 ---
# Tkinter, shutil, os, time, glob는 Python 내장 라이브러리
# Selenium, webdriver-manager 설치 필요: pip install selenium webdriver-manager
# pywin32 설치 필요 (Windows에서 다른 창 최소화 기능 사용 시)

# --- 다운로드 폴더 생성 확인 ---
if not os.path.exists(DOWNLOAD_DIR):
    try:
        os.makedirs(DOWNLOAD_DIR)
        print(f"다운로드 폴더를 생성했습니다: {DOWNLOAD_DIR}")
    except OSError as e:
        messagebox.showerror("오류", f"다운로드 폴더 생성 실패: {e}\n프로그램을 종료합니다.")
        exit()

# --- 파일 다운로드 완료 대기 헬퍼 함수 ---
def wait_for_download_completion(download_path, timeout=60):
    """
    지정된 경로에서 새로운 파일이 나타나고 다운로드가 완료될 때까지 기다립니다.
    가장 최근에 수정된 파일이 다운로드 중인 파일이라고 가정합니다.
    """
    print(f"다운로드 폴더 '{download_path}'에서 새로운 파일 다운로드를 기다립니다... (최대 {timeout}초)")
    seconds = 0
    last_size = -1
    file_path = None
    start_time = time.time()

    existing_files = {os.path.join(download_path, f): os.path.getctime(os.path.join(download_path, f))
                      for f in os.listdir(download_path) if os.path.isfile(os.path.join(download_path, f))}


    while time.time() - start_time < timeout:
        time.sleep(1)
        seconds += 1

        list_of_files = [os.path.join(download_path, f) for f in os.listdir(download_path) if os.path.isfile(os.path.join(download_path, f))]

        if not list_of_files:
             continue

        new_files = [(f, os.path.getctime(f)) for f in list_of_files if f not in existing_files]

        if new_files:
            latest_new_file, latest_ctime = max(new_files, key=lambda item: item[1])

            if not latest_new_file.endswith('.crdownload'):
                 file_path = latest_new_file
                 current_size = os.path.getsize(file_path)

                 if current_size > 0 and current_size == last_size and seconds > 2:
                     print(f"파일 크기 안정됨: {current_size} bytes. 다운로드 완료 추정.")
                     return file_path
                 else:
                     last_size = current_size

    print(f"다운로드 완료 대기 시간 초과 ({timeout}초). 새로운 파일이 감지되지 않았거나 다운로드가 완료되지 않았습니다.")
    return None

# --- 다운로드된 파일 처리 헬퍼 함수 ---
def process_downloaded_file(downloaded_file_path, target_name):
    """다운로드된 파일을 복사하고 원하는 이름으로 변경하여 저장합니다."""
    if not downloaded_file_path or not os.path.exists(downloaded_file_path):
        print("다운로드된 파일을 찾을 수 없습니다.")
        return False

    target_file_name = f"{target_name}.xlsx"
    target_file_path = os.path.join(DOWNLOAD_DIR, target_file_name)

    print(f"다운로드된 원본 파일: {os.path.basename(downloaded_file_path)}")
    print(f"복사본 대상 파일명: {target_file_name}")

    # 기존에 같은 이름의 대상 파일이 있으면 삭제
    if os.path.exists(target_file_path):
        print(f"기존 대상 파일 삭제 시도: {target_file_name}")
        try:
            os.remove(target_file_path)
            print("기존 파일 삭제 완료.")
        except OSError as e:
            print(f"기존 대상 파일 삭제 실패: {e}")
            messagebox.showerror("오류", f"기존 파일 삭제 실패: {target_file_name}\n{e}\n파일이 사용 중인지 확인해 보세요.")
            return False

    # 다운로드된 파일을 원하는 이름으로 복사
    try:
        shutil.copy2(downloaded_file_path, target_file_path)
        print(f"파일 복사 완료: {os.path.basename(downloaded_file_path)} -> {target_file_name}")
        return True
    except OSError as e:
        print(f"파일 복사 실패: {e}")
        messagebox.showerror("오류", f"파일 복사 실패: {os.path.basename(downloaded_file_path)} -> {target_file_name}\n{e}")
        return False

# --- 다운로드 실행 함수 (KB) ---

def run_kb_monthly_download():
    """KB 월간 시계열 다운로드 실행"""
    driver = None
    downloaded_file_path = None

    try:
        # Chrome 옵션 설정
        chrome_options = webdriver.ChromeOptions()
        prefs = {
            "download.default_directory": DOWNLOAD_DIR,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing.enabled": True
        }
        chrome_options.add_experimental_option("prefs", prefs)

        # 제작 과정 확인을 위해 백그라운드 실행 (headless) 주석 처리 유지
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        print("KB 월간 시계열 다운로드 시작...")
        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

        print(f"페이지 접속: {KB_BASE_URL}")
        driver.get(KB_BASE_URL)

        # 페이지 로딩 및 초기화 대기 (3초)
        print(f"페이지 로딩 및 초기화 대기 ({INITIAL_LOAD_WAIT}초)...")
        time.sleep(INITIAL_LOAD_WAIT)

        # 다운로드 버튼 클릭
        try:
            print(f"다운로드 버튼 찾기 및 클릭 시도: {DOWNLOAD_BUTTON_XPATH}")
            download_button = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, DOWNLOAD_BUTTON_XPATH))
            )
            download_button.click()
            print("다운로드 버튼 클릭 완료.")

        except (NoSuchElementException, TimeoutException) as e:
            print(f"오류: 다운로드 버튼({DOWNLOAD_BUTTON_XPATH})을 찾거나 클릭할 수 없습니다. 페이지 로딩 문제 또는 XPath 오류 가능성.\n{e}")
            messagebox.showerror("오류", f"다운로드 버튼을 찾거나 클릭할 수 없습니다.\n페이지 구조를 확인해 보세요.\n{e}")
            return
        except Exception as e:
            print(f"오류: 다운로드 버튼 클릭 중 예상치 못한 오류 발생: {e}")
            messagebox.showerror("오류", f"다운로드 버튼 클릭 중 오류 발생.\n{e}")
            return

        # 파일 다운로드 완료 대기
        print("파일 다운로드 완료 대기...")
        downloaded_file_path = wait_for_download_completion(DOWNLOAD_DIR, timeout=45)

        if downloaded_file_path:
            print("다운로드된 파일 확인됨. 파일 처리 시작...")
            success = process_downloaded_file(downloaded_file_path, "월간시계열")
            if success:
                 messagebox.showinfo("완료", "KB 월간 시계열 다운로드 완료 ")
            else:
                 pass
        else:
            messagebox.showerror("오류", "KB 월간 시계열 다운로드 중 시간 초과 또는 파일 감지 실패.\nKB 웹사이트의 다운로드를 수동으로 확인해보세요.")

    except WebDriverException as e:
         print(f"오류: WebDriver 관련 오류 발생. 크롬 드라이버 문제 또는 브라우저 문제 가능성: {e}")
         messagebox.showerror("WebDriver 오류", f"웹 브라우저 제어 중 오류 발생.\n크롬 드라이버나 브라우저 상태를 확인하세요.\n{e}")
    except Exception as e:
        print(f"오류: KB 월간 시계열 다운로드 중 예상치 못한 전체 오류 발생: {e}")
        messagebox.showerror("알 수 없는 오류", f"KB 월간 시계열 다운로드 중 오류 발생: {e}")

    finally:
        if driver:
            driver.quit()
            print("브라우저 종료.")
        print("KB 월간 시계열 다운로드 프로세스 종료.")


def run_kb_weekly_download():
    """KB 주간 시계열 다운로드 실행"""
    driver = None
    downloaded_file_path = None

    try:
        # Chrome 옵션 설정
        chrome_options = webdriver.ChromeOptions()
        prefs = {
            "download.default_directory": DOWNLOAD_DIR,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing.enabled": True
        }
        chrome_options.add_experimental_option("prefs", prefs)

        # 제작 과정 확인을 위해 백그라운드 실행 (headless) 주석 처리 유지
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        print("KB 주간 시계열 다운로드 시작...")
        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

        print(f"페이지 접속: {KB_BASE_URL}")
        driver.get(KB_BASE_URL)

        # 페이지 로딩 및 초기화 대기 (3초)
        print(f"페이지 로딩 및 초기화 대기 ({INITIAL_LOAD_WAIT}초)...")
        time.sleep(INITIAL_LOAD_WAIT)

        # '주간시계열' 탭 클릭
        try:
            print(f"주간시계열 탭 찾기 및 클릭 시도: {KB_WEEKLY_TAB_XPATH}")
            weekly_tab = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, KB_WEEKLY_TAB_XPATH))
            )
            weekly_tab.click()
            print("주간시계열 탭 클릭 완료.")
            # 탭 내용 로딩 및 다운로드 버튼 활성화 대기 (3초)
            print(f"탭 내용 로딩 및 다운로드 버튼 활성화 대기 ({TAB_LOAD_WAIT}초)...")
            time.sleep(TAB_LOAD_WAIT)

        except (NoSuchElementException, TimeoutException) as e:
            print(f"오류: 주간시계열 탭({KB_WEEKLY_TAB_XPATH})을 찾거나 클릭할 수 없습니다. 페이지 로딩 문제 또는 XPath 오류 가능성.\n{e}")
            messagebox.showerror("오류", f"주간시계열 탭을 찾거나 클릭할 수 없습니다.\n**주간시계열 탭의 XPath를 확인해 보세요.**\n{e}")
            return
        except Exception as e:
            print(f"오류: 주간시계열 탭 클릭 중 예상치 못한 오류 발생: {e}")
            messagebox.showerror("오류", f"주간시계열 탭 클릭 중 오류 발생.\n{e}")
            return


        # 다운로드 버튼 클릭
        try:
            print(f"다운로드 버튼 찾기 및 클릭 시도: {DOWNLOAD_BUTTON_XPATH}")
            download_button = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, DOWNLOAD_BUTTON_XPATH))
            )
            download_button.click()
            print("다운로드 버튼 클릭 완료.")
        except (NoSuchElementException, TimeoutException) as e:
             print(f"오류: 다운로드 버튼({DOWNLOAD_BUTTON_XPATH})을 찾거나 클릭할 수 없습니다. 탭 로딩 문제 또는 XPath 오류 가능성.\n{e}")
             messagebox.showerror("오류", f"다운로드 버튼을 찾거나 클릭할 수 없습니다.\n페이지 구조를 확인해 보세요.\n{e}")
             return
        except Exception as e:
            print(f"오류: 다운로드 버튼 클릭 중 예상치 못한 오류 발생: {e}")
            messagebox.showerror("오류", f"다운로드 버튼 클릭 중 오류 발생.\n{e}")
            return


        # 파일 다운로드 완료 대기
        print("파일 다운로드 완료 대기...")
        downloaded_file_path = wait_for_download_completion(DOWNLOAD_DIR, timeout=45)

        if downloaded_file_path:
            print("다운로드된 파일 확인됨. 파일 처리 시작...")
            success = process_downloaded_file(downloaded_file_path, "주간시계열")
            if success:
                 messagebox.showinfo("완료", "KB 주간 시계열 다운로드 완료 ")
            else:
                 pass
        else:
             messagebox.showerror("오류", "KB 주간 시계열 다운로드 중 시간 초과 또는 파일 감지 실패.\nKB 웹사이트의 다운로드를 수동으로 확인해보세요.")

    except WebDriverException as e:
         print(f"오류: WebDriver 관련 오류 발생. 크롬 드라이버 문제 또는 브라우저 문제 가능성: {e}")
         messagebox.showerror("WebDriver 오류", f"웹 브라우저 제어 중 오류 발생.\n크롬 드라이버나 브라우저 상태를 확인하세요.\n{e}")
    except Exception as e:
        print(f"오류: KB 주간 시계열 다운로드 중 예상치 못한 전체 오류 발생: {e}")
        messagebox.showerror("알 수 없는 오류", f"KB 주간 시계열 다운로드 중 오류 발생: {e}")

    finally:
        if driver:
            driver.quit()
            print("브라우저 종료.")
        print("KB 주간 시계열 다운로드 프로세스 종료.")


# --- 다운로드 실행 함수 (R-ONE - 추후 구현) ---
def run_rone_weekly_download():
    messagebox.showinfo("정보", "R-ONE 주간 시계열 다운로드 기능은 아직 준비 중입니다.")
    print("R-ONE 주간 시계열 다운로드 기능은 추후 구현 예정입니다.")

def run_rone_monthly_download():
    messagebox.showinfo("정보", "R-ONE 월간 시계열 다운로드 기능은 아직 준비 중입니다.")
    print("R-ONE 월간 시계열 다운로드 기능은 추후 구현 예정입니다.")

# --- 다른 창 최소화 함수 (Windows 전용) ---
def minimize_other_windows(my_window_handle):
    if not IS_WINDOWS:
        print("경고: 다른 창 최소화 기능은 Windows에서만 지원됩니다.")
        return

    def enum_windows_callback(hwnd, lParam):
        # 창이 보이는지 확인
        if win32gui.IsWindowVisible(hwnd):
            # 창 제목이 있는지 확인 (숨겨진 창이나 바탕화면 등 제외)
            if win32gui.GetWindowText(hwnd) != '':
                # 현재 프로그램 창이 아닌 경우 최소화
                if hwnd != my_window_handle:
                    try:
                        win32gui.ShowWindow(hwnd, win32con.SW_MINIMIZE)
                        # print(f"창 최소화: {win32gui.GetWindowText(hwnd)}") # 어떤 창이 최소화되는지 확인하고 싶다면 주석 해제
                    except Exception as e:
                        pass # 특정 창 최소화 실패는 무시

        return True # 계속 열거

    print("다른 창들을 최소화합니다...")
    win32gui.EnumWindows(enum_windows_callback, 0)
    print("다른 창 최소화 시도 완료.")


# --- GUI 구성 ---
root = tk.Tk()

# 1. 창을 화면에 나타나기 전에 일단 숨깁니다.
root.withdraw() # 창 숨기기

# 화면 중앙 계산 및 창 위치 설정 (숨겨진 상태에서도 크기 정보는 유효합니다)
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

center_x = int((screen_width / 2) - (WINDOW_WIDTH / 2))
center_y = int((screen_height / 2) - (WINDOW_HEIGHT / 2))

# 창 크기와 위치 설정 (숨겨진 상태에 적용)
root.geometry(f"{WINDOW_WIDTH}x{WINDOW_HEIGHT}+{center_x}+{center_y}")
root.title("다운로더") # 창 제목 설정
root.configure(bg=BG_COLOR) # 창 배경색 설정

# 2. 다른 창들을 최소화합니다.
if IS_WINDOWS:
    minimize_other_windows(root.winfo_id())
    # 다른 창 최소화 명령이 처리될 시간을 아주 짧게 기다립니다.
    time.sleep(0.1) # 100 밀리초 대기

# 3. 숨겼던 저희 프로그램 창을 다시 보이게 하고 맨 앞으로 가져옵니다.
root.deiconify() # 창 다시 보이게 하기
root.lift()      # 창을 맨 앞으로 가져오기 (선택 사항이지만 중앙에 나타난 창이 다른 창 뒤에 숨겨지는 것을 방지)


# 메인 컨텐츠 프레임 (좌우 분할)
main_frame = tk.Frame(root, bg=BG_COLOR)
main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10) # 창 전체 영역 사용

# 좌측 KB 프레임
kb_frame = tk.Frame(main_frame, bg=BG_COLOR, padx=10, pady=10, bd=2, relief="groove")
kb_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5)

kb_label = tk.Label(kb_frame, text="KB시세", bg=BG_COLOR, font=('맑은 고딕', 10, 'bold'))
kb_label.pack(pady=(0, 10)) # 위쪽 여백 0, 아래쪽 여백 10

kb_weekly_button = tk.Button(kb_frame, text="주간시계열 다운로드", command=run_kb_weekly_download)
kb_weekly_button.pack(pady=5, fill=tk.X) # y축 여백 5, x축으로 확장

kb_monthly_button = tk.Button(kb_frame, text="월간시계열 다운로드", command=run_kb_monthly_download)
kb_monthly_button.pack(pady=5, fill=tk.X)

# 우측 R-ONE 프레임
rone_frame = tk.Frame(main_frame, bg=BG_COLOR, padx=10, pady=10, bd=2, relief="groove")
rone_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=5)

rone_label = tk.Label(rone_frame, text="R-ONE시세", bg=BG_COLOR, font=('맑은 고딕', 10, 'bold'))
rone_label.pack(pady=(0, 10))

rone_weekly_button = tk.Button(rone_frame, text="주간시계열 다운로드", command=run_rone_weekly_download)
rone_weekly_button.pack(pady=5, fill=tk.X)

rone_monthly_button = tk.Button(rone_frame, text="월간시계열 다운로드", command=run_rone_monthly_download)
rone_monthly_button.pack(pady=5, fill=tk.X)

# GUI 이벤트 루프 시작
root.mainloop()


pywin32 라이브러리를 성공적으로 로드했습니다.
DPI 인식 설정을 시도했습니다.
다른 창들을 최소화합니다...
다른 창 최소화 시도 완료.
KB 주간 시계열 다운로드 시작...
페이지 접속: https://kbland.kr/webview.html#/main/statistics?channel=kbland
페이지 로딩 및 초기화 대기 (3초)...
주간시계열 탭 찾기 및 클릭 시도: //*[@id="__BVID__30___BV_tab_button__"]
주간시계열 탭 클릭 완료.
탭 내용 로딩 및 다운로드 버튼 활성화 대기 (3초)...
다운로드 버튼 찾기 및 클릭 시도: //*[@id="reference2"]/div[1]/button
다운로드 버튼 클릭 완료.
파일 다운로드 완료 대기...
다운로드 폴더 'C:\Bi시각화\시계열'에서 새로운 파일 다운로드를 기다립니다... (최대 45초)
파일 크기 안정됨: 19631021 bytes. 다운로드 완료 추정.
다운로드된 파일 확인됨. 파일 처리 시작...
다운로드된 원본 파일: 20250728_주간시계열.xlsx
복사본 대상 파일명: 주간시계열.xlsx
기존 대상 파일 삭제 시도: 주간시계열.xlsx
기존 파일 삭제 완료.
파일 복사 완료: 20250728_주간시계열.xlsx -> 주간시계열.xlsx
브라우저 종료.
KB 주간 시계열 다운로드 프로세스 종료.
